In [27]:
import pandas as pd
import os
import numpy as np

from tqdm import tqdm
from glob import glob

In [28]:
# Captura o diretório atual e salva na variável
path = diretorio = os.getcwd() + '/'
print(path)

/home/karem/Artigo_Periodico/YoloV5/


In [29]:
'''
carrega o arquivo contendo o tamanho original das imagens
'''

df = pd.read_csv('/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_1/test.csv')

# coloca a extensão .png no final do nome da imagem
#df['id'] = df['id'].apply(lambda x: x.replace('_image', '.png'))
#df[['id']] = df[['id']] + '.png'

### Remove os casos negativos
indexNames = df[df['class_name'] == 'negative'].index
# Delete these row indexes from dataFrame
df.drop(indexNames, inplace = True)


# reinicia o index
df = df.reset_index(drop=True)


df['class_name'].loc[(df['class_name'] == 'typical')] = 'covid'
df['class_name'].loc[(df['class_name'] == 'indeterminate')] = 'covid'

#df['new_class'].loc[(df['class_name'] == 'negative')] = 'normal'

print(len(df))
df.head(5)
#df

391


/home/karem/anaconda3/envs/env_py39/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,e6d2c043ff7b_image,covid,640,640,[181.0],[196.0],[289.0],[526.0],[108.0],[330.0]
1,65b2468a585d_image,covid,640,640,"[41.0, 320.0]","[127.0, 172.0]","[202.0, 472.0]","[285.0, 319.0]","[161.0, 152.0]","[158.0, 147.0]"
2,f9729a4e5c44_image,covid,640,640,[30.0],[325.0],[121.0],[506.0],[91.0],[181.0]
3,562962571ea3_image,covid,640,640,"[346.0, 102.0]","[165.0, 225.0]","[590.0, 289.0]","[474.0, 509.0]","[244.0, 188.0]","[310.0, 284.0]"
4,f5020d2a27d1_image,covid,640,640,"[112.0, 375.0]","[130.0, 357.0]","[246.0, 565.0]","[512.0, 499.0]","[134.0, 190.0]","[382.0, 142.0]"


### Funções

In [30]:
# Convert lista para string
def convert(lst):
    i = ''.join(lst)
    str1 = re.sub(r"[\([{})\]]", "", i)
    return str1

In [31]:
#Cria o dietório caso não exista
def create_folders(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [32]:
# se houver a pasta, ela é removida
import shutil
def remove_folders(path):
    if os.path.exists(path):
        shutil.rmtree(path)

In [33]:
# Converte string para lista
def Convert_str_to_int(string):
    li = list(string.split(", "))
    return li

In [34]:
import re
def convert_df_lista_int(str1):
    #str1 = df['xmin_ori'][1]
    str1 = re.sub(r"[\([{})\]]", "", str1) # remove os caracteres da string
    #str1 = (str1.replace('.0', '')) # substirui a casa decimal .0 por vazio, tornando a string no formato int
    str1 = Convert_str_to_int(str1) # converte a string em lista
    #str1 = list(map(float, str1)) # converte a lista de string em float
    #str1 = list(map(int, str1)) # converte a lista de float em int
    str1 = [int(float(i)) for i in str1] # convert os valores para inteiro e depois string    
    return str1

### Create files -> GT

In [ ]:
# # Gera os labels do GT

# #path
# path_labelsGT = '/home/karem/Artigo_SPIE_Kaggle/YoloEvaluation/input/ground_truth/' ----------

# remove_folders(path_labelsGT)
# create_folders(path_labelsGT)

# cont_label = 0
# for i in range(len(df)):
    
#     # cria um txt com os nomes das imagens
#     # substitui o .png do dataframe para txt
#     nome_file = os.path.basename(df.iloc[i,0].replace('_image', '.txt'))
    
#     # cria o arquivo txt no modo escrita (w)
#     file = open(path_labelsGT + nome_file, "w") 
# #     if not (os.path.exists(path_labels_pred + nome_file)): # se o arquivo txt não existir (se a yolo nõa gerou label)
# #         print(nome_file)
# #         file2 = open(path_labels_pred + nome_file, "w")
# #         file2.write('0 0 0 0 0 0') # salva no txt     
    
#     lista_int_x1 = convert_df_lista_int(df['xmin'][i])
#     lista_int_x2 = convert_df_lista_int(df['xmax'][i])
#     lista_int_y1 = convert_df_lista_int(df['ymin'][i])
#     lista_int_y2 = convert_df_lista_int(df['ymax'][i])
        
#     # se a classe for covid, salvará como 0 (0 = covid)
#     if df['class_name'][i] == 'covid':
#         for j in range(len(lista_int_x1)):
#             bb = [lista_int_x1[j], lista_int_y1[j], lista_int_x2[j], lista_int_y2[j]]
#             #print(bb)
#             result = (convert(str(bb)).replace(',', '')) # transforma a lista em string
#             #file.write('0 ' + result + '\n') # salva no txt       
#             cont_label += 1
            
#     file.close() # fecha o txt
# #     file2.close()
    
# print(cont_label)

### Create files -> Pred (Yolo)

In [35]:
# Convert lista para string
def convert2(lst):    
    return ' '.join(lst)

In [36]:
# função que volta o tamanho do bounding box para orginal
# o tamanho salvo foi normalizado entre 0 e 1
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [37]:
image_ids = []
PredictionStrings = []
path_labels_yolo = path + 'yolov5/runs/detect/exp/labels/*.txt'

path_labels_pred = path + 'yolov5/runs/detect/exp/labels_formated/'
remove_folders(path_labels_pred)
create_folders(path_labels_pred)

for file_path in tqdm(glob(path_labels_yolo)):    
    image_id = file_path.split('/')[-1].split('.')[0] + '_image'
    
    w, h = df[['width', 'height']].values[0]
    
    for i in range(len(df)):
        if image_id == df['id'][i]:
            #img_width = int(df['width_ori'][i])
            #img_height = int(df['height_ori'][i])
            w = int(df['width'][i])
            h = int(df['height'][i])
            break
    
    f = open(file_path, 'r')    
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
       
    
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
     
    # cria um novo txt com os tamanhos originais dos BB
    nome_file = os.path.basename(file_path)
    file = open(path_labels_pred + nome_file, "w")     
    for i in range(int(len(bboxes)/6)):
        result = convert2(bboxes[i*6:i*6+6])
        file.write(result + '\n')
        #print(result)
        #print(bboxes[i*6:i*6+6])
    file.close() 
       
    
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

    pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})
pred_df


100%|████████████████████████████████████████████████████████████████████████████████| 390/390 [00:00<00:00, 884.85it/s]


,id,PredictionString
0,705eea360dd8_image,0 0.4 413 420 545 573 0 0.4 111 184 244 550 0 ...
1,692f1f18f529_image,0 0.5 117 86 240 446 0 0.6 427 104 602 458 0 0...
2,e46f7573012c_image,0 0.5 142 94 256 353 0 0.5 410 181 568 456 0 0...
3,2a7181500a00_image,0 0.5 413 183 580 538 0 0.6 127 193 315 555 0 ...
4,0d17348ba547_image,0 0.3 108 153 259 354 0 0.4 436 210 555 381 0 ...
...,...,...
385,b98174b36c85_image,0 0.2 62 108 196 294 0 0.2 395 319 508 527 0 0...
386,7317e56f9189_image,0 0.3 417 210 505 398 0 0.3 63 166 184 470 0 0...
387,b1c963b7c90a_image,0 0.2 292 280 451 483 0 0.4 295 98 459 500 0 0...
388,e1ae4be3ef0c_image,0 0.3 61 228 183 561 0 0.3 61 228 183 561
